In [1]:
# 1. Generate a starting structure from a SMILES string using `protomol`
from protomol import smiles

smi = "C[CH2]"  # SMILES formula for propyl radical

# Generate a geometry for the SMILES formula
geo = smiles.geometry(smi)

In [2]:
# 2. Optimize the structure with GFN2-xTB using `qcop`
import qcio
import qcop
from protomol import qc

struc = qc.struc.from_geometry(geo)
prog_inp = qcio.ProgramInput(
    structure=struc, calctype="optimization", model={"method": "gfn2"}
)
prog_out = qcop.compute("crest", prog_inp)

equilibrium_structure = prog_out.results.final_structure

In [ ]:
# 3. By displacing the equilibrium structure and running energy calculations,
# calculate the Hessian of the PES at equilibrium.
import numpy
import functools
from protomol import geom
from numpy.typing import ArrayLike


def coordinate_count(geo: geom.Geometry) -> int:
    """Count number of coordinates in geometry.

    (Generic function that could be added to protomol.geom.)

    :param geo: Geometry
    :return: Number of coordinates
    """
    return geom.count(geo) * 3


def displace(geo: geom.Geometry, disp: ArrayLike) -> geom.Geometry:
    """Displace geometry by arbitrary coordinate vector.

    (Generic function that could be added to protomol.geom.)

    :param geo: Geometry
    :param disp: Displacement vector
    :return: Displaced geometry
    """
    geo = geo.model_copy()
    natms = geom.count(geo)
    geo.coordinates = geo.coordinates + numpy.reshape(disp, (natms, 3))
    return geo


def step_displace_coordinate(
    geo: geom.Geometry, idx: int, step: int = 1, step_size: float = 0.005
) -> geom.Geometry:
    """Step-displace geometry coordinate.

    (Project-specific helper function that could be added to util.py)

    :param geo: Geometry
    :param idx: Flat coordinate index, 0, 1, ..., 3 * n
    :param step: Number of steps to displace (positive or negative)
    :param step_size: Displacement step size
    :return: Geometry
    """
    ncoord = coordinate_count(geo)
    disp = numpy.eye(ncoord, ncoord)[idx] * step * step_size
    return displace(geo, disp)


def hessian(
    prog: str, ene_inp: qcio.ProgramInput, step_size: float = 0.005
) -> numpy.ndarray:
    # Get initial input and reference geometry
    ene_inp0 = ene_inp.model_copy()
    geo0 = qc.struc.geometry(ene_inp0.structure)

    # Define a function to calculate the energy of a given displacement
    @functools.cache
    def energy(idx: int = 0, step: int = 0, idx2: int = 0, step2: int = 0):
        print(
            f"Calculating energy for idx1={idx}, step1={step}, idx2={idx2}, step2={step2}"
        )
        # Generate displaced geometry
        geo = step_displace_coordinate(geo0, idx=idx, step=step, step_size=step_size)
        geo = step_displace_coordinate(geo, idx=idx2, step=step2, step_size=step_size)
        # Copy energy input and replace with displaced geometry
        ene_inp = ene_inp0.model_copy(update={"structure": qc.struc.from_geometry(geo)})
        ene_out = qcop.compute(prog, ene_inp)
        return ene_out.results.energy

    # Create an empty Hessian matrix
    ncoord = coordinate_count(geo)
    hess = numpy.zeros((ncoord, ncoord))

    # Calculate diagonal elements of Hessian matrix
    for idx in range(ncoord):
        hess[idx, idx] = (
            energy(idx=idx, step=+1) + energy(idx=idx, step=-1) - 2 * energy()
        ) / (step_size**2)

    # Calculate off-diagonal elements of Hessian matrix
    # (Iterate over lower triangle to avoid )
    # Then, calculate off-diagonals (iterate over lower triangle)
    for idx1 in range(ncoord):
        for idx2 in range(idx1):
            hess[idx1, idx2] = hess[idx2, idx1] = (
                energy(idx=idx1, step=+1, idx2=idx2, step2=+1)
                + energy(idx=idx1, step=-1, idx2=idx2, step2=-1)
                - energy(idx=idx1, step=+1)
                - energy(idx=idx1, step=-1)
                - energy(idx=idx2, step=+1)
                - energy(idx=idx2, step=-1)
                + 2 * energy()
            ) / (2 * step_size**2)

    return hess


ene_inp = qcio.ProgramInput(
    structure=equilibrium_structure, calctype="energy", model={"method": "gfn2"}
)
hess = hessian("crest", ene_inp=ene_inp)
print(hess)

Calculating energy for idx1=0, step1=1, idx2=0, step2=0
Calculating energy for idx1=0, step1=-1, idx2=0, step2=0
Calculating energy for idx1=0, step1=0, idx2=0, step2=0
Calculating energy for idx1=1, step1=1, idx2=0, step2=0
Calculating energy for idx1=1, step1=-1, idx2=0, step2=0
Calculating energy for idx1=2, step1=1, idx2=0, step2=0
Calculating energy for idx1=2, step1=-1, idx2=0, step2=0
Calculating energy for idx1=3, step1=1, idx2=0, step2=0
Calculating energy for idx1=3, step1=-1, idx2=0, step2=0
Calculating energy for idx1=4, step1=1, idx2=0, step2=0
Calculating energy for idx1=4, step1=-1, idx2=0, step2=0
Calculating energy for idx1=5, step1=1, idx2=0, step2=0
Calculating energy for idx1=5, step1=-1, idx2=0, step2=0
Calculating energy for idx1=6, step1=1, idx2=0, step2=0
Calculating energy for idx1=6, step1=-1, idx2=0, step2=0
Calculating energy for idx1=7, step1=1, idx2=0, step2=0
Calculating energy for idx1=7, step1=-1, idx2=0, step2=0
Calculating energy for idx1=8, step1=1, 

In [7]:
geo = geom.Geometry(
    symbols=["O", "H", "H"],
    coordinates=[[0, 0, 0], [1, 0, 0], [0, 1, 0]]
)
geo

Geometry(symbols=['O', 'H', 'H'], coordinates=array([[0., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]]), charge=0, spin=0)

In [4]:
# 4. Check your answer by calculating the same Hessian directly with `qcop`.
prog_inp = qcio.ProgramInput(
    structure=equilibrium_structure, calctype="hessian", model={"method": "gfn2"}
)
prog_out = qcop.compute("crest", prog_inp)

hessian = prog_out.results.hessian
print(hessian)

[[ 4.7095171e-01  2.2784160e-02 -1.2219600e-03 -2.5140207e-01
   1.0234190e-02  1.3663350e-02 -6.5026230e-02  2.8110450e-02
  -6.3665950e-02 -4.7928530e-02  2.3282090e-02  4.1416650e-02
  -7.6795500e-02 -8.3911460e-02  7.5291600e-03 -1.5519770e-02
  -1.5654100e-03  1.1651200e-03 -1.4272540e-02  9.6275000e-04
   1.0360200e-03]
 [ 2.2784160e-02  5.1256117e-01 -1.2814230e-02 -2.1584100e-03
  -6.8693540e-02  4.6363500e-03  3.4292950e-02 -8.5285830e-02
   8.0699640e-02  3.1423160e-02 -1.0745294e-01 -9.4245280e-02
  -8.3488020e-02 -2.5029271e-01  2.0980400e-02 -2.4293700e-02
  -1.5047000e-04  2.2376700e-03  2.1487250e-02 -6.1479000e-04
  -1.4427600e-03]
 [-1.2219600e-03 -1.2814230e-02  4.3522458e-01  1.3339110e-02
   3.8232800e-03 -3.7790550e-02 -7.2044620e-02  8.0714060e-02
  -1.8347983e-01  5.0657700e-02 -9.3542740e-02 -1.7008919e-01
   7.7722900e-03  2.1121530e-02 -4.7122780e-02  3.8637600e-03
  -2.4706000e-04 -2.0841000e-03 -2.3379100e-03  9.6401000e-04
   5.3979400e-03]
 [-2.5140207e-01